In [ ]:
from datetime import date
from itertools import product

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

from app.model import DispatchConfig
from run_dispatch import run_dispatch

In [ ]:
# DATES = pd.date_range(
#     start="2024-05-01",
#     end="2024-05-31",
#     inclusive="both"
# )
DATES = [
    pd.Timestamp("2024-01-01"),
    pd.Timestamp("2024-10-13"),
    pd.Timestamp("2024-04-28"),
    pd.Timestamp("2024-03-29"),
    pd.Timestamp("2024-01-07"),
    pd.Timestamp("2024-06-09"),
    pd.Timestamp("2024-09-30"),
]
CONFIGS = [
    "preideal",
    # "bess_preideal",
    "bess_preideal_resource",
]

BESS_VALUES = {
    "BESS_1": {
        "MWh_nom": 10000,
        "hours_to_deplete": 2,
        "efficiency": 0.9,
        "min_soc": 0.05,
        "max_soc": 0.98,
        "initial_soc": 0.05,
        "charge_bid": 0,
        "discharge_bid": 0,
    },
}


In [ ]:

output_unstacked = pd.DataFrame()
for (date_, dispatch_type) in product(DATES,CONFIGS):
    BESS = BESS_VALUES if "bess" in dispatch_type.lower() else None
    mpo = run_dispatch(DispatchConfig(dispatch_type=dispatch_type), DISPATCH_DATE=date_.date(), show_figs=True, BESS=BESS)
    # mpo["dispatch_type"] = dispatch_typex
    output_unstacked = pd.concat([output_unstacked, mpo])
output = output_unstacked.stack().reset_index()
output.columns = ["datetime", "source", "MPO"]
output.drop_duplicates(inplace=True)

In [ ]:
output

In [ ]:
mapper = {
    "MPO preideal Modelo - DERs None": "MPO Modelo",
    "MPO bess_preideal_resource Modelo - DERs None": "MPO SAEB recurso",
}

output["source"] = output["source"].apply(lambda x: mapper.get(x,x))
output_ = output[output["source"].isin(list(mapper.values()))]

In [ ]:
output

In [ ]:
from plotly.subplots import make_subplots

# Create subplots: 2 columns and 4 rows, with the last row spanning both columns
fig = make_subplots(
    rows=4, cols=2,
    specs=[[{}, {}],
           [{}, {}],
           [{}, {}],
           [{"colspan": 2}, None]],
    subplot_titles=[str(date.date()) for date in DATES]
)

# Add traces to the subplots
for i, (date, serie) in enumerate(output_.groupby(output_["datetime"].dt.date)):
    row = i // 2 + 1
    col = i % 2 + 1
    showlegend = True if i == 0 else False
    for trace in px.line(serie, x="datetime", y="MPO", color="source").data:
        trace.showlegend = showlegend
        fig.add_trace(trace, row=row, col=col)

# Update layout
fig.update_layout(
    height=1200, width=800,
    showlegend=True,
    # xaxis=dict(showgrid=True, dtick=3_600_000,),

)

fig.show()

In [ ]:
with pd.option_context("display.max_rows", None):
    display(output[output["datetime"].dt.date == DATES[0].date()])

In [ ]:
output_unstacked = output.pivot(index="datetime", columns="source", values="MPO")

In [ ]:
# Group by days and apply the mean_absolute_percentage_error function
grouped = output_unstacked.groupby(output_unstacked.index.date).apply(
    lambda x: mean_absolute_percentage_error(y_pred=x["MPO preideal Modelo - DERs None"], y_true=x["MPO preideal XM"])
)

# Convert the result to a DataFrame for better readability
grouped_df = pd.DataFrame(grouped, columns=["MAPE %"])
grouped_df["MAPE %"] = grouped_df["MAPE %"] * 100
grouped_df

In [ ]:
output_unstacked.to_csv("differences_for_month_.csv")

In [ ]:
fig = go.Figure()

# Add line for "MPO preideal Modelo"
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['MPO preideal Modelo'],
    mode='lines',
    name='MPO preideal Modelo'
))

# Add line for "MPO bess_preideal Modelo"
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['MPO bess_preideal_resource Modelo'],
    mode='lines',
    name='MPO bess_preideal Modelo',
    fill='tonexty',
    line_color='green'
))

# Add area for positive differences
# fig.add_trace(go.Scatter(
#     x=output_unstacked.index,
#     y=output_unstacked['Difference'].apply(lambda x: x if x > 0 else 0),
#     fill='tozeroy',
#     mode='none',
#     fillcolor='rgba(0, 255, 0, 0.5)',  # Green color for positive differences
#     name='Positive Difference',
#     line_shape='hv',
#     yaxis='y2'
# ))

# # Add area for negative differences
# fig.add_trace(go.Scatter(
#     x=output_unstacked.index,
#     y=output_unstacked['Difference'].apply(lambda x: x if x < 0 else 0),
#     fill='tozeroy',
#     mode='none',
#     fillcolor='rgba(255, 0, 0, 0.5)',  # Red color for negative differences
#     name='Negative Difference',
#     line_shape='hv',
#     yaxis='y2'
# ))

fig.update_layout(
    title='MPO preideal Modelo vs MPO bess_preideal Modelo',
    xaxis_title='Datetime',
    yaxis_title='MPO [COP/MWh]',
    yaxis2=dict(
        title='Difference [COP/MWh]',
        overlaying='y',
        side='right'
    ),
    legend_title='Legend'
)

fig.show()

In [ ]:
output_unstacked.columns